In [1]:
# !pip install torch

In [2]:
# Regular imports
import matplotlib.pyplot as plt
import torch

from torch import nn

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
data_path = "/kaggle/input/majortry6/labeled_data.csv"

In [5]:
import numpy as np
import pickle
import pandas as pd
pd.options.mode.copy_on_write = True

data = pd.read_csv(data_path)
df = pd.DataFrame(data)
print(df.columns)
df.columns = ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'classk', 'tweet']

Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'class', 'tweet'],
      dtype='object')


In [6]:
df["classk"] = df["classk"].map({2:1, 1:0, 0:0}) 
# label = 0 = hate
# label = 1 = not_hate

In [7]:
tweets = df.tweet
res = []

for i in range(len(tweets)):
    splitt = tweets[i].split(":")
    if len(splitt) == 1:
        res.append(splitt[0])
    else:
        res.append(splitt[-1])
        
tweets = res

# print(tweets)

In [8]:
hate_words = ['Moist', 'Cunt', 'Panties', 'Fuck', 'Hate', 'Nigger', 'Pussy', 'Ass', 'Motherfucker', 'Bitch', 'Damn']
not_hate_words = ['Love', 'Peace', 'Kindness', 'Happiness', 'Respect', 'Friendship',
'Appreciation', 'Hope', 'Encouragement', 'Support', 'Caring']

In [9]:

from transformers import AutoTokenizer
from transformers import BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer("I love Pokemon")
# # tokenizer.decode(tokenizer("I love Pokemon")['input_ids'])
# from transformers import BertTokenizer, BertForSequenceClassification
# import torch
# from torch.utils.data import DataLoader, TensorDataset

# # Load pre-trained model and tokenizer
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize and prepare data
# input_texts = ["your input sentences"]
# labels = [0, 1, ...]  # corresponding labels

# tokenized_inputs = tokenizer(input_texts, truncation=True, padding=True, return_tensors="pt")

# # Create DataLoader
# dataset = TensorDataset(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'], torch.tensor(labels))
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Set up optimizer and loss function
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# loss_fn = torch.nn.CrossEntropyLoss()

# # Fine-tuning loop
# for epoch in range(num_epochs):
#     for batch in dataloader:
#         inputs, attention_mask, target = batch
#         outputs = model(inputs, attention_mask=attention_mask)
#         loss = loss_fn(outputs.logits, target)
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

# # Evaluate on a test set
# # (Prepare the test set similarly and use model.eval() before inference)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 1045, 2293, 20421, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [10]:
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello my dog is cute", return_tensors="pt")
print(inputs)

outputs = model(**inputs)

print(len(outputs.last_hidden_state[0])) # => no. of tokens + [CLS] + [SEP]
print(len(outputs.last_hidden_state[0][1])) # => hidden dim size
print(len(outputs.last_hidden_state[0][2])) # => hidden dim size

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  7592,  2026,  3899,  2003, 10140,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
7
768
768


In [11]:
temp = []
model = BertModel.from_pretrained("bert-base-uncased")

for word in hate_words:
    inputs = tokenizer(word, return_tensors="pt")
#     print(inputs)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])], dim=0)
    temp.append(outputs.tolist())

# temp = temp[1:]
# print(hate_space)
# print(len(temp)) # => no. of words = no. of rows
# print(len(temp[0])) # => no. of columns in each row = embedding vector size

# hate_space = temp.clone().detach().requires_grad_(True)
hate_space =torch.tensor(temp).to(device).requires_grad_()

hate_space.is_leaf
# hate_space
# hate_space.requires_grad

True

In [12]:
temp = []
model = BertModel.from_pretrained("bert-base-uncased")

# with torch.no_grad():
for word in not_hate_words:
    inputs = tokenizer(word, return_tensors="pt")
#     print(inputs)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])], dim=0)
    temp.append(outputs.tolist())

# temp = temp[1:]
# print(hate_space)
# print(len(temp)) # => no. of words = no. of rows
# print(len(temp[0])) # => no. of columns in each row = embedding vector size


not_hate_space = torch.tensor(temp).to(device).requires_grad_()

print(not_hate_space.is_leaf)

# not_hate_space
# not_hate_space.requires_grad

True


In [13]:
# Trial matmul

t1 = torch.randn(11, 768)
t2 = torch.randn(4, 23, 768)

print(torch.transpose(t2, 1, 2).shape)

torch.matmul(t1, torch.transpose(t2, 1, 2)).shape

torch.Size([4, 768, 23])


torch.Size([4, 11, 23])

In [14]:
class Custom_Dataset(torch.utils.data.Dataset):
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __getitem__(self, index: int) -> (torch.Tensor, int): # => (X, y)
        
        X = self.data[index]
        y = self.labels[index]

        return (X, y)
    
    
    def __len__(self) -> int:
        
        return len(self.data)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets, list(df.classk), test_size=0.2, shuffle=True)

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

19826 19826
4957 4957


In [16]:
train_data = Custom_Dataset(data=X_train, labels=y_train)
test_data = Custom_Dataset(data=X_test, labels=y_test)

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=5, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [17]:
import random
l=[[random.uniform(-1,1)] for i in range(11)]
l

[[-0.5802978135671393],
 [-0.111940953049384],
 [3.120319963212381e-05],
 [-0.4309275228783529],
 [0.35625580850130145],
 [0.0699453614819543],
 [0.42173486468192656],
 [0.692872812317141],
 [0.17590740660716953],
 [0.41641280748276666],
 [-0.5909710878445542]]

In [18]:
# next(iter(test_loader)) 
tensor_shape = (11,1)
hate_space_weights = torch.tensor(l).to(device).requires_grad_()
not_hate_space_weights = torch.tensor(l).to(device).requires_grad_()

print(hate_space_weights.device,not_hate_space_weights.is_leaf)

cuda:0 True


In [19]:
hate_space_weights.is_leaf

True

In [20]:
import torch.nn.functional as F

class Model0(nn.Module):
    
    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model
    
    def forward(self, X):
        inputs = tokenizer(X, return_tensors="pt")
        inputs.to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state.mean(1).transpose(0,1)
        col_norms = torch.norm(outputs, p=2, dim=0, keepdim=True)

        outputs = outputs / col_norms
        norm_hate_space = hate_space / torch.norm(hate_space, p=2, dim=1, keepdim=True)
        norm_not_hate_space = not_hate_space / torch.norm(not_hate_space, p=2, dim=1, keepdim=True)
        Psk_hate = torch.matmul(torch.matmul(norm_hate_space, outputs).transpose(0, 1), hate_space_weights)#.requires_grad_()
        Psk_not_hate = torch.matmul(torch.matmul(norm_not_hate_space, outputs).transpose(0, 1), not_hate_space_weights)#.requires_grad_()

        # Construct the output tensor with requires_grad=True
        output_tensor = torch.tensor([Psk_hate, Psk_not_hate]).requires_grad_()
        print(output_tensor.shape)
        return output_tensor


In [21]:
model0 = Model0(model, tokenizer)

# X, y = next(iter(train_loader))
# Psk_hate, Psk_not_hate = model0(X)

# Semi-SSM

In [22]:
def L_inter(spaces):
    
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    
    loss = torch.tensor(0.0, requires_grad=True)
    
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        
        loss = loss + cur
    
    return loss
            
            

def L_intra(spaces):
    loss=torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss=loss+1/torch.var(spaces[k], dim=0)
    
    return torch.sum(loss)/loss.shape[0]




In [23]:
L_inter(torch.concat((torch.tensor([[0.33, 0.54, 0.37], [0.72, 0.74, 0.31], [0.61, 0.26, 0.56]], dtype=torch.float32), 
       torch.tensor([[0.22, 0.32, 0.15], [0.12, -0.08, 0.45], [0.14, 0.28, -0.11]], dtype=torch.float32))))

tensor(33.6994, grad_fn=<AddBackward0>)

In [24]:
# temp = torch.tensor(10.3, requires_grad=True)

# temp

In [25]:
# res = [temp, temp]

# loss = 0

# loss += res[0] * res[1]

# loss

input1 = torch.randn(100, 1)
input2 = torch.randn(100, 1)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)
output

In [26]:
print(hate_space,not_hate_space)

tensor([[ 0.8009,  0.2601,  0.2316,  ...,  0.0455,  0.0118, -0.0370],
        [ 0.1845, -0.0866,  0.1542,  ..., -0.2302, -0.3558, -0.2348],
        [ 0.6341,  0.0180, -0.3330,  ...,  0.1851, -0.3067, -0.4023],
        ...,
        [ 0.8014,  0.2637,  0.2895,  ...,  0.1221,  0.1728, -0.4732],
        [ 0.5409,  0.4092, -0.2325,  ...,  0.1790, -0.2709, -0.0346],
        [ 0.5508,  0.1983,  0.0521,  ...,  0.3905, -0.0753, -0.5880]],
       device='cuda:0', requires_grad=True) tensor([[ 0.5933,  0.2685, -0.0267,  ..., -0.4845, -0.4192, -0.3698],
        [ 0.2493,  0.1343, -0.1606,  ..., -0.1305, -0.3765, -0.3898],
        [ 0.0551, -0.1996, -0.1785,  ..., -0.0520, -0.1908, -0.2057],
        ...,
        [ 0.2502, -0.2729, -0.2597,  ...,  0.1696, -0.5105, -0.0785],
        [ 0.2900, -0.2058, -0.1541,  ..., -0.0185, -0.5686,  0.0864],
        [ 0.2823, -0.0846, -0.1393,  ...,  0.1534, -0.2074, -0.2804]],
       device='cuda:0', requires_grad=True)


In [27]:
print(hate_space_weights,not_hate_space_weights)

tensor([[-5.8030e-01],
        [-1.1194e-01],
        [ 3.1203e-05],
        [-4.3093e-01],
        [ 3.5626e-01],
        [ 6.9945e-02],
        [ 4.2173e-01],
        [ 6.9287e-01],
        [ 1.7591e-01],
        [ 4.1641e-01],
        [-5.9097e-01]], device='cuda:0', requires_grad=True) tensor([[-5.8030e-01],
        [-1.1194e-01],
        [ 3.1203e-05],
        [-4.3093e-01],
        [ 3.5626e-01],
        [ 6.9945e-02],
        [ 4.2173e-01],
        [ 6.9287e-01],
        [ 1.7591e-01],
        [ 4.1641e-01],
        [-5.9097e-01]], device='cuda:0', requires_grad=True)


In [28]:
model0.to(device)
# hate_space = hate_space.to(device)
# not_hate_space = not_hate_space.to(device)
# hate_space_weights = hate_space_weights.to(device)
# not_hate_space_weights = not_hate_space_weights.to(device)
print(hate_space.is_leaf)
optimizer = torch.optim.Adam(params=[hate_space, not_hate_space, hate_space_weights, not_hate_space_weights], lr=0.01)
model0.train()
criterion = nn.CrossEntropyLoss()
model0.train()
# with torch.inference_mode():
for _ in range(1):
    for i, batch in enumerate(train_loader):
        if i == 2:
            break
        if i % 1000 == 0:
            print(i)

        x, Y = batch
        loss = 0

        for X, y in zip(x, Y):
            y_preds = model0(X)

            # Use long instead of float for target labels
            loss += F.cross_entropy(y_preds.to(device), torch.tensor([1-y, y]).to(device).float())

        loss /= len(x)
        loss += L_inter(torch.cat((hate_space, not_hate_space))) + L_intra(torch.stack([hate_space, not_hate_space]))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(not_hate_space_weights.grad, hate_space.grad, loss.item())




True
0
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
None tensor([[-2.0692e-02, -6.1839e-03, -1.8119e-02,  ...,  1.2480e-03,
         -1.1703e-02, -2.0455e-02],
        [ 2.8541e-02,  1.1966e-02, -1.3492e-02,  ...,  3.7211e-02,
          6.1739e-03, -6.8660e-03],
        [-7.3706e-03,  6.4916e-03,  1.5660e-02,  ..., -1.6960e-02,
          3.7917e-03,  4.6497e-03],
        ...,
        [-2.0725e-02, -6.3600e-03, -2.1569e-02,  ..., -8.7267e-03,
         -1.9520e-02,  9.5350e-03],
        [ 7.4797e-05, -1.3994e-02,  9.6476e-03,  ..., -1.6170e-02,
          2.0494e-03, -2.0622e-02],
        [-7.1412e-04, -2.9447e-03, -7.3754e-03,  ..., -4.3758e-02,
         -7.4649e-03,  1.7418e-02]], device='cuda:0') 548.548095703125
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
None tensor([[-0.0189, -0.0058, -0.0163,  ...,  0.0024, -0.0110, -0.0184],
        [ 0.0258,  0.0110, -0.0123,  ...,  0.0335,  0.0058, -0.0066],
        [-0.0072,

In [29]:
print(hate_space_weights,not_hate_space_weights)

tensor([[-5.8030e-01],
        [-1.1194e-01],
        [ 3.1203e-05],
        [-4.3093e-01],
        [ 3.5626e-01],
        [ 6.9945e-02],
        [ 4.2173e-01],
        [ 6.9287e-01],
        [ 1.7591e-01],
        [ 4.1641e-01],
        [-5.9097e-01]], device='cuda:0', requires_grad=True) tensor([[-5.8030e-01],
        [-1.1194e-01],
        [ 3.1203e-05],
        [-4.3093e-01],
        [ 3.5626e-01],
        [ 6.9945e-02],
        [ 4.2173e-01],
        [ 6.9287e-01],
        [ 1.7591e-01],
        [ 4.1641e-01],
        [-5.9097e-01]], device='cuda:0', requires_grad=True)


In [30]:
print(hate_space,not_hate_space)

tensor([[ 0.8208,  0.2801,  0.2516,  ...,  0.0258,  0.0318, -0.0170],
        [ 0.1645, -0.1066,  0.1741,  ..., -0.2501, -0.3758, -0.2148],
        [ 0.6541, -0.0020, -0.3529,  ...,  0.2051, -0.3267, -0.4223],
        ...,
        [ 0.8214,  0.2837,  0.3094,  ...,  0.1421,  0.1927, -0.4931],
        [ 0.5228,  0.4292, -0.2525,  ...,  0.1990, -0.2909, -0.0146],
        [ 0.5705,  0.2183,  0.0720,  ...,  0.4105, -0.0553, -0.6080]],
       device='cuda:0', requires_grad=True) tensor([[ 0.6133,  0.2884, -0.0069,  ..., -0.5044, -0.4391, -0.3897],
        [ 0.2293,  0.1542, -0.1407,  ..., -0.1505, -0.3964, -0.4098],
        [ 0.0352, -0.2196, -0.1585,  ..., -0.0325, -0.1709, -0.1871],
        ...,
        [ 0.2302, -0.2928, -0.2797,  ...,  0.1896, -0.5305, -0.0585],
        [ 0.2700, -0.2257, -0.1342,  ...,  0.0015, -0.5885,  0.1063],
        [ 0.2623, -0.1046, -0.1195,  ...,  0.1734, -0.1875, -0.3003]],
       device='cuda:0', requires_grad=True)


In [31]:
torch.concat((hate_space, not_hate_space)).shape

torch.Size([22, 768])

In [32]:
# correct = 0
# total = len(y_test)

# true = []
# preds = []

# with torch.inference_mode():
#     for i, batch in enumerate(test_loader):
#         if i % 1000 == 0:
#             print(i)
#         X, y = batch
#         Psk_hate, Psk_not_hate = model0(X)

#         y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))

#         true.extend(y)
#         preds.append(y_predict)

#         cur = torch.eq(y_predict, y)

#         correct += torch.sum(cur)

# print(f"Test Accuracy: {correct/total}")
# print(f"Test Accuracy: {torch.sum(torch.eq(torch.tensor(true), torch.tensor(preds))) / total}")       

In [33]:
# from sklearn.metrics import precision_score, recall_score

# print(precision_score(y_test, preds, average='macro'))
# print(recall_score(y_test, preds, average='macro'))